In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
# read in 13F results
file_name = '/content/gdrive/My Drive/capstone/full database/Results_20160630_20200630.csv'
df = pd.read_csv(file_name)
df['HOLDDATE'] = df['HOLDDATE'].str.replace(r' 00:00:00', '')

In [ ]:
n_investors = df.LGCYINVESTORID.nunique()
print('There are {} investors in 13F data.'.format(n_investors))

In [ ]:
# read in list of investors of interest
df_investors = pd.read_csv('/content/active_investor_list.csv')
# df_investors = df_investors[df_investors.isActive==True]

ls_investors = df_investors.LgcyInvestorId.to_list()
print('There are {} investors which are we interested in.'.format(len(ls_investors)))

In [ ]:
active_ls_investors = df_investors[df_investors.isActive==True]

In [ ]:
# get those in predefined list
df_sub = df.loc[df['LGCYINVESTORID'].isin(ls_investors),['INSTRID', 'COMNAME', 'LGCYINVESTORID', 'FULLNAME', 'HOLDDATE', 'SHSHLDVAL']]

# calculate the sum of SHSHLDVAL for each investor in each quarter
df_sub['SHSHLDVAL_Sum_ByQuarter'] = df_sub.groupby(['LGCYINVESTORID','HOLDDATE']).transform('sum').drop(['INSTRID', 'FULLNAME'], axis = 1)

# calculate the (13F only) pct of the SHSHLDVAL of each instrument in its investor's sum of SHSHLDVAL for each quarter 
# (i.e. revised version of 'PCTPORTFOLIO', so that sum up to 1)
df_sub['PCTPORTFOLIO_13F'] = df_sub['SHSHLDVAL'] / df_sub['SHSHLDVAL_Sum_ByQuarter']

In [ ]:
sum(df_sub.drop_duplicates(subset=['INSTRID','COMNAME']).COMNAME.isna())

In [ ]:
df_sub.drop_duplicates(subset=['INSTRID','COMNAME'])[df_sub.drop_duplicates(subset=['INSTRID','COMNAME']).COMNAME.isna()]

In [ ]:
sum(df_sub.drop_duplicates(subset=['INSTRID','COMNAME']).COMNAME.isna())/len(df_sub.drop_duplicates(subset=['INSTRID','COMNAME']))

In [ ]:
df_sub

In [ ]:
vanguard_id = '2004260'

## Fill in missing time

In [ ]:
def fill_in_missing_date(df, dateColName, valColName):
  '''
  df: Column for investor id is 'LGCYINVESTORID' and column for instrumnet id is 'INSTRID'
  dateColName: Fill in all the dates starting from the earlist to latest mentioned in this column
  valColName: The column you want to fill in missing value.
              The missing value will be filled as 0.

  Return: a dataframe with filled missing value as 0 in valColName
  '''
  # create pivot table with id and date
  df_pivot = pd.pivot_table(df,index='LGCYINVESTORID', columns= dateColName, values=valColName)
  # fill in the missing value as 0. Each investor have data for all time stamp.
  df_pivot.fillna(0,inplace=True)

  # turn the pivot table to original format
  df_stack = df_pivot.stack().reset_index()
  df_stack.rename(columns = {0: valColName},inplace=True)
  # map back the original df
  df = pd.merge(df.drop(columns=valColName),df_stack,on = ['LGCYINVESTORID',dateColName])
  return df_stack


## Feature: Asset allocation


In [ ]:
asset_aloc = pd.read_csv('/content/gdrive/My Drive/capstone/full database/AssetAllocWithDesc.csv')

In [ ]:
asset_aloc[asset_aloc.LgcyInvestorId.isin(ls_investors)]

In [ ]:
asset_aloc[asset_aloc.LgcyInvestorId.isin(ls_investors)]['TotAstAlloc'].value_counts(normalize=True)

In [ ]:
asset_aloc['EqAstAlloc'].value_counts()

In [ ]:
asset_aloc['TotAstAlloc'].value_counts(normalize=True)

In [ ]:
asset_aloc[asset_aloc['TotAstAlloc'] == 'AARANGE1']

In [ ]:
own2code = pd.read_excel('/content/gdrive/My Drive/capstone/full database/Own2Code.xlsx')

In [ ]:
own2code[own2code['Type_'] == 11]

## Feature: pctportofolio with aggregated market cap

In [ ]:
mktcap = pd.read_csv('/content/gdrive/My Drive/capstone/full database/MktCap_MktCapSize_20160630_20200630.csv')
mktcap

In [ ]:
merge = pd.merge(df_sub, mktcap, how='inner', left_on=['INSTRID', 'HOLDDATE'], right_on=['InstrId', 'EffectDate'])
merge.drop(columns=['InstrId', 'EffectDate'])

agg_mktcap_pivot = pd.pivot_table(merge,index = ['LGCYINVESTORID', 'HOLDDATE'], columns = ['MktCapSize'],values=['PCTPORTFOLIO_13F'],aggfunc=[np.sum],fill_value=0)

investor_set = set(agg_mktcap_pivot.index.get_level_values('LGCYINVESTORID'))
holddate_set = set(agg_mktcap_pivot.index.get_level_values('HOLDDATE'))

mktcap_cat = ['PctLargeCap', 'MegaCap', 'MicroCap', 'MidCap', 'NanoCap', 'SmallCap']

In [ ]:
mktcap

In [ ]:
pd.merge(df_sub, mktcap,left_on=['INSTRID', 'HOLDDATE'], right_on=['InstrId', 'EffectDate'])

In [ ]:
agg_mktcap_pivot

In [ ]:
agg_mktcap_pivot.columns = mktcap_cat
agg_mktcap_pivot = agg_mktcap_pivot.reset_index()
for inv in investor_set:
  holddates = set(agg_mktcap_pivot[agg_mktcap_pivot['LGCYINVESTORID']==inv]['HOLDDATE'].to_list())
  if len(holddates) < len(holddate_set):
    # print('Investor ID:', inv, ' missing:')
    # print(holddate_set - holddates)
    missing_dates = holddate_set - holddates
    for md in missing_dates:
      agg_mktcap_pivot = agg_mktcap_pivot.append([{'LGCYINVESTORID': inv, 'HOLDDATE':md}])
      

In [ ]:
agg_mktcap_pivot = agg_mktcap_pivot.fillna(0)

In [ ]:
assert len(agg_mktcap_pivot) == len(investor_set) * len(holddate_set)

In [ ]:
agg_mktcap_pivot = agg_mktcap_pivot.set_index(['LGCYINVESTORID','HOLDDATE']).stack().reset_index()
agg_mktcap_pivot = agg_mktcap_pivot.rename(columns={agg_mktcap_pivot.columns[-2]:'MktCap',agg_mktcap_pivot.columns[-1]:'PCTPORTFOLIO_13F'})

In [ ]:
agg_mktcap_pivot.columns

In [ ]:
agg_mktcap_pivot = agg_mktcap_pivot.pivot_table(index=['LGCYINVESTORID'],columns=['HOLDDATE', 'MktCap'],values=['PCTPORTFOLIO_13F'])
agg_mktcap_pivot.columns =  agg_mktcap_pivot.columns.droplevel()

## Feature: Turnover

In [ ]:
# read in turnover rate
turnover = pd.read_csv('/content/gdrive/My Drive/capstone/full database/Own2InvHldTO.csv')

In [ ]:
# select these in a list
turnover = turnover[turnover.LgcyInvestorId.isin(ls_investors) & (turnover.CalcBasisCode==2)]

In [ ]:
# fill in 0 for missing date
turnover_pivot = turnover.pivot_table(index='LgcyInvestorId', columns='EffectToDate',values='OwnTurnover').fillna(0)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
# create distance matrix for gephi 
turnover_matrix = pd.DataFrame(euclidean_distances(turnover_pivot),index = turnover_pivot.index, columns=turnover_pivot.index)
# turnover_matrix.to_csv('/content/gdrive/My Drive/capstone/distance matrix/turnover_distance.csv')

## Feature: Top 20% pct

Definition: the porportion of investment on top 20% of instruments.
This represents the concentration of the investment. In other words, if this score is low, the investor prefers  distributed investment. Otherwise, the investor prefers concentrated investment.

### Create features for the listed investors

In [ ]:
def topPct(df, pct=0.2 ,k=10):
  n = len(df)
  k = int(pct * n)
  if k == 0:
    k = 1
  return sum(sorted(df)[-k:])/sum(df)

df_toppct = df_sub.groupby(['LGCYINVESTORID','FULLNAME','HOLDDATE']).agg({'SHSHLDVAL': topPct}).reset_index()

df_toppct.rename(columns={'SHSHLDVAL':'TOP20%SHSHLDVALpct'},inplace=True)

In [ ]:
df_toppct.LGCYINVESTORID.nunique()

In [ ]:
df_toppct = fill_in_missing_date(df_toppct, 'HOLDDATE', 'TOP20%SHSHLDVALpct')

In [ ]:
# df_toppct.to_csv('/content/gdrive/My Drive/capstone/full database/TOP20%SHSHLDVALpct_20160630_20200630.csv',index=False)

### Compute Euclidean distance

In [ ]:
concentration_pivot = df_toppct.pivot_table(index='LGCYINVESTORID', columns='HOLDDATE',values='TOP20%SHSHLDVALpct').fillna(0)

In [ ]:
# create distance matrix for listed investor
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
concentration_matrix = pd.DataFrame(euclidean_distances(concentration_pivot),index =concentration_pivot.index,columns=concentration_pivot.index)

In [ ]:
# concentration_matrix.to_csv('conventration_distance.csv')

## Feature: investment style, Industry, market of Investors' contacts

### Get description of different specialized code

We need to map with Own2Code(Type_=13) to get the description of the SpecCode.
SpecCat:
  - SpecCap == 1: Industry
    - Code in Own2Code: IS (201 different)
    - Code in data: xx
    - **Problem: IS193,IS194,IS195,IS196**
  - SpecCap == 3: Market Capitalization range in which the Investor Contact specializes
    - Code in Own2Code: MCS1 - MCS4
    - Code in data: 224 225 226 227
  - SpecCap == 4: Investment Style 
    - Code in Own2Code: ISS1 - ISS20(ISS19 not exists)
    - Code in data: 201 - 220 
    - **Problem: ISS19 not exists**

In [ ]:
# read in contacts info for investors in 13F
contact_info = '/content/gdrive/My Drive/capstone/full database/ContactResults.csv'
contact_df = pd.read_csv(contact_info)

In [ ]:
contact_df.head()

In [ ]:
# read in code table
code_file_name = '/content/gdrive/My Drive/capstone/full database/Own2Code.xlsx'
code_df = pd.read_excel(code_file_name)
code_df = code_df.fillna('9999999')
spec_code_df = code_df[code_df.Type_ == 13]

In [ ]:
def explanation_spec(SpecCat, SpecCode):
  if pd.isna(SpecCat) or pd.isna(SpecCode):
    return 
  SpecCode = int(SpecCode)
  try:
    if SpecCat == 1:
      return 'Industry_' + spec_code_df[spec_code_df.Code == ('IS'+str(SpecCode))] ['Desc_'].values[0]
    elif SpecCat == 3:
      return 'Market_' + spec_code_df[spec_code_df.Code == ('MCS'+str(SpecCode-223))]['Desc_'].values[0]
    elif SpecCat == 4:
      return 'InvStyle_' + spec_code_df[spec_code_df.Code == ('ISS'+str(SpecCode-200))]['Desc_'].values[0]
  except:
    No_explanation.append((SpecCat, SpecCode))
    return 'No_explanation'

In [ ]:
No_explanation = []
contact_df['SpecDesc_'] = contact_df.apply(lambda x: explanation_spec(x.SpecCat, x.SpecCode), axis=1)

In [ ]:
# part of specCat don't have specDesc_
from collections import Counter
Counter(No_explanation)

In [ ]:
# Remove those with no explanation
contact_df = contact_df[contact_df['SpecDesc_'] != 'No_explanation']

### Aggregate for each investor

In [ ]:
investor_style = pd.pivot_table(contact_df,index=['LgcyInvestorId'],columns=['SpecCat','SpecDesc_'], values=['LgcyPersonId'],aggfunc='count')
# investor_style.fillna(0,inplace=True)
investor_style.columns = investor_style.columns.droplevel()

In [ ]:
print('{}({}) investors in 13F have contact info.'.format(len(investor_style), len(investor_style)/n_investors))

In [ ]:
# contact_df.groupby(['LgcyInvestorId','SpecCat','SpecDesc_']).agg({'LgcyPersonId':'count'}).reset_index().to_csv('investstyle.csv')

### Create features for the listed investors

In [ ]:
investor_style_sub = investor_style[investor_style.index.isin(ls_investors)]

In [ ]:
# whether we need to do td-idf
investor_style_sub[4].sum(axis=0).hist()

In [ ]:
investor_style_sub[4].sum(axis=0).sort_values()

In [ ]:
# compute weight, use idf
industry_weight = np.log(len(investor_style_sub) / (investor_style_sub[1].count(axis=0) + 1))
invStyle_weight = np.log(len(investor_style_sub) / (investor_style_sub[4].count(axis=0) + 1))

In [ ]:
# normalize
def normalize(df):
  return df.div(df.sum(axis=1),axis=0).fillna(0)

industry_sub = normalize(investor_style_sub[1])
invStyle_sub = normalize(investor_style_sub[4])

In [ ]:
industry_sub

In [ ]:
# compute distance 
invStyle_matrix = pd.DataFrame(euclidean_distances(industry_sub),index =industry_sub.index,columns=industry_sub.index)

In [ ]:
# invStyle_matrix.to_csv('invStyle_distance.csv')

## Feature: Number of instruments

In [ ]:
n_instruments = df_sub.groupby(['LGCYINVESTORID','HOLDDATE']).agg({'INSTRID':'count'}).reset_index()
n_instruments_pivot = pd.pivot_table(n_instruments,index=['LGCYINVESTORID'],columns=['HOLDDATE'],values=['INSTRID']).fillna(0)

In [ ]:
n_instruments_pivot.columns = n_instruments_pivot.columns.droplevel()

In [ ]:
n_instruments_pivot_static = n_instruments.groupby('LGCYINVESTORID').agg({'INSTRID':'max'})

## Feature: Total Asset

In [ ]:
total_asset = df_sub.groupby(['LGCYINVESTORID','HOLDDATE']).agg({'SHSHLDVAL_Sum_ByQuarter':'max'}).reset_index()
n_instruments_pivot = pd.pivot_table(df_sub,index=['LGCYINVESTORID'],columns=['SHSHLDVAL_Sum_ByQuarter'],values=['INSTRID']).fillna(0)

In [ ]:
total_asset = df_sub.groupby(['LGCYINVESTORID','HOLDDATE']).agg({'SHSHLDVAL_Sum_ByQuarter':'max'}).reset_index()
total_asset_pivot = pd.pivot_table(total_asset,index=['LGCYINVESTORID'],columns=['HOLDDATE'],values=['SHSHLDVAL_Sum_ByQuarter']).fillna(0)

In [ ]:
total_asset_pivot.columns = total_asset_pivot.columns.droplevel()

In [ ]:
total_asset_pivot_static = total_asset.groupby('LGCYINVESTORID').agg({'SHSHLDVAL_Sum_ByQuarter':'max'})

## K_means

In [ ]:
agg_mktcap_pivot.head(1)

In [ ]:
# one company no turnover
turnover_pivot.head(1)

In [ ]:
concentration_pivot.head(1)

In [ ]:
industry_sub.head(1)

In [ ]:
# normalize turnover_pivot
max_turnover = turnover_pivot.max().max()
turnover_pivot_standard = turnover_pivot/max_turnover

In [ ]:
# normalize number of instruments
max_n_instruments = n_instruments_pivot.max().max()
n_instruments_pivot_standard = n_instruments_pivot/max_n_instruments
n_instruments_pivot_static_standard = n_instruments_pivot_static/max_n_instruments

In [ ]:
# normalize total asset
max_total_asset = total_asset_pivot.max().max()
total_asset_pivot_standard = total_asset_pivot/max_total_asset
total_asset_pivot_static_standard = total_asset_pivot_static/max_total_asset

In [ ]:
X = pd.concat([agg_mktcap_pivot,
               turnover_pivot_standard, 
               n_instruments_pivot_static_standard,
               total_asset_pivot_static_standard,
               concentration_pivot,invStyle_sub], axis=1).fillna(0)
              
X_sub = X[X.index.isin(active_ls_investors.LgcyInvestorId)]

In [ ]:
draw_tsne(X_sub)

In [ ]:
from sklearn.cluster import KMeans

def fit_kmeans(n_components,X):
    km = KMeans(n_components)
    km.fit(X)
    predictions = km.predict(X)
    return predictions

In [ ]:
from sklearn.manifold import TSNE

def draw_tsne(X, predictions=None):
    tsne = TSNE(n_components=2)
    x = tsne.fit_transform(X)
    if predictions is None:
      plt.scatter(x[:,0],x[:,1])
      plt.title('pctportfolio clustering')
    else:
      plt.scatter(x[:,0],x[:,1],c=predictions)
      plt.title('pctportfolio clustering')      

In [ ]:
from sklearn import metrics
from sklearn.cluster import SpectralClustering

def InternalEvaluation(data, clusters):
    scores = {}
    """
    The score is bounded between -1 for incorrect clustering and +1 for highly dense clustering. 
    Scores around zero indicate overlapping clusters.
    The score is higher when clusters are dense and well separated, which relates to a standard concept of a cluster.
    """
    scores['_silhouette_score'] =metrics.silhouette_score(data,clusters ,metric='euclidean')
    """
    The score is higher when clusters are dense and well separated, which relates to a standard concept of a cluster.
    The score is fast to compute
    """
    scores['_calinski_harabaz_score'] = metrics.calinski_harabasz_score(data,clusters)
    """
    Zero is the lowest possible score. Values closer to zero indicate a better partition.
    The Davies-Boulding index is generally higher for convex clusters than other concepts of clusters, 
    such as density based clusters like those obtained from DBSCAN.
    """
    scores['_davies_bouldin_score'] = metrics.davies_bouldin_score(data,clusters)
    return scores

def draw_scores(scores, start_k=2):
  score_type = list(scores[0].keys())
  n_type = len(score_type)
  plt.figure(figsize=(8,4))
  for i in range(n_type):
    sc_type = score_type[i]
    score = [sc[sc_type] for sc in scores]
    plt.subplot(1, n_type, i+1)
    plt.plot(range(start_k,len(score) + start_k ), score)
    plt.title(' '.join(sc_type.split('_')).title())
    plt.xlabel('k')
  plt.suptitle('Metrics for Clustering')

In [ ]:
def modeling(X):
  scores = []
  for k in range(2, 10):
    predictions = fit_kmeans(k,X)
    score = InternalEvaluation(X, predictions)
    scores.append(score)
  plt.figure(1)
  draw_scores(scores)
  plt.figure(2)
  # draw_tsne(X)

In [ ]:
agg_mktcap_pivot,
              #  turnover_pivot, 
               concentration_pivot,invStyle_sub

In [ ]:
modeling(X_sub)


In [ ]:
predictions = fit_kmeans(3,X_sub)
draw_tsne(X_sub,predictions)